# Post-processing of case study for central plant results

## Usage notes

All figures will be written to the directory `img` as pdf and png files.

I run this notebook on Ubuntu, using this version of matplotlib:
```
pip freeze | grep matplotlib
matplotlib==3.5.1
```
Some older versions return an error because they do not support some plot configurations.
I set in the virtual machine the RAM to 12GB as 8GB is not sufficient to parse two annual result files.

## Import required libraries

In [1]:
import cases as cas
import importlib
import post_process_configurations as con
importlib.reload(con)


<module 'post_process_configurations' from '/home/mwetter/test/thermal-grid-jba-main/PythonResources/RunCases/post_process_configurations.py'>

## Read annual results

This section also clears the old results to free up memory.

In [2]:
# Free up storage, then read new data
import gc
gc.collect()

print("Cleaned up memory.")

Cleaned up memory.


Read result file. This takes around 3 minutes per result file.

In [3]:
#list_of_variables = ['ETot.y',
#                     "CPUtime"]
#r_base=get_partial_results('base', list_of_variables)
print("Reading result files")
r_base=con.get_results('base')
print("Read base results")
r_base_hBor_0_8=con.get_results('base_hBor_0.8')
print("Read base_hBor_0.8 results")
r_base_hBor_1_2=con.get_results('base_hBor_1.2')
print("Read base_hBor_1.2 results")
r_base_dDis_0_8=con.get_results('base_dDis_0.8')
print("Read base_hBor_0.8 results")
r_base_dDis_1_2=con.get_results('base_dDis_1.2')
print("Read base_hBor_1.2 results")
r_base_noEco=con.get_results('base_noEco')
print("Read base_noEco results")

Reading result files
Read base results
Read base_hBor_0.8 results
Read base_hBor_1.2 results
Read base_hBor_0.8 results
Read base_hBor_1.2 results
Read base_noEco results


In [4]:
AFlo = r_base.max('datDis.AFlo')

print(f"Base case: Total energy use: {r_base.max('ETot.y')/3600/1E9:.2f} GWh")
print(f"Base case: Total energy use: {r_base.max('ETot.y')/3600/1E9/AFlo*1e6:.2f} kWh/(m a)")

print(f"Base case: Total energy cost: {r_base.max('totEleCos.y')/1E6:.2f} million USD per year")
print(f"Base case: Total energy cost: {r_base.max('totEleCos.y')/AFlo:.2f} USD/(m2 a)")



Base case: Total energy use: 23.43 GWh
Base case: Total energy use: 209.19 kWh/(m a)
Base case: Total energy cost: 3.46 million USD per year
Base case: Total energy cost: 30.87 USD/(m2 a)


In [5]:
print(f"CPUtime, base {r_base.max('CPUtime')/3600.:.2f} h")

CPUtime, base 13.73 h


### Write capacities to LaTeX file

In [6]:
# Note that the unyt package used in this function does not allow to reload the module.
# Hence we put the function in its own file
##import post_process_write_latex_table as lat
##lat.write_latex_capacity_table(r_base)

### Energy use


In [7]:
casesToCompare = [r_base, r_base_hBor_0_8, r_base_hBor_1_2, r_base_dDis_0_8, r_base_dDis_1_2, r_base_noEco]
labelsOfCasesToCompare = ['base', '$0.8 \, h_{bor}$', '$1.2 \, h_{bor}$', '$0.8 \, d_{dis}$', '$1.2 \, d_{dis}$', 'no economizer']
fileNamesOfCasesToCompare = ['base', 'base_hBor_0_8', 'base_hBor_1_2', 'base_dDis_0_8', 'base_dDis_1_2', 'base_no_eco']
con.plot_energy(casesToCompare, labelsOfCasesToCompare)


All electricity use = [209.1947678  213.3192148  205.97338947 263.05361203 193.97411303
 209.08805569]
Sum of plot = [209.19476585 213.3192135  205.97339337 263.05361333 193.97412408
 209.0880609 ]

Heat pumps in ETS   & 5.21 &  46.5 \\
Heat pumps in plant & 3.24 &  29.0 \\
Pumps               & 5.23 &  46.7 \\
Fans                & 0.89 &  8.0 \\
Non-HVAC electricity for buildings & 8.85 &  79.1  \\ \hline
Total & 23.43 &  209.2 \\ \hline


## Loop temperatures

In [8]:
importlib.reload(con)
con.plot_loop_temperatures(casesToCompare, fileNamesOfCasesToCompare)


# Plot plant operation

In [9]:
# List of days to plot
days = [
    {
        "xlim": [31, 32],
        "date": "Feb. 1",
        "name": "Winter"
    },
    {
        "xlim": [160, 161],
        "date": "June 10",
        "name": "Spring"
    },
    {
        "xlim": [213, 214],
        "date": "Aug. 2",
        "name": "Summer"
    }
]
# List of variables to plot for each subplot
lis = [
        {
            "y_label": "Controls",
            "y_lim": [-3, 3],
            "factor": 1,
            "offset": 0,
            "vars": [
                {
                    "label": "$y_{st}$",
                    "var": "cenPla.gen.ind.ySt",
                    "linewidth": 1,
                    },
                    {
                    "label": "$y_{pla}$",
                    "var": "cenPla.gen.ind.yPlaOut",
                    }
                ]
        },
        {
            "y_label": "Temperature\n[$^\\circ$C]",
            "y_lim": [10, 28],
            "factor": 1,
            "offset": -273.15,
            "vars": [
                 {
                    "label": "$T_{pla,hea,set}$",
                    "var": "cenPla.gen.ind.TActPlaHeaSet",
                    "linewidth": 1,
                    "marker": ">",
                    "color": "r",
                    "skip_if_ySea": 3
                    },
                 {
                    "label": "$T_{pla,coo,set}$",
                    "var": "cenPla.gen.ind.TActPlaCooSet",
                    "linewidth": 1,
                    "marker": ">",
                    "color": "r",
                    "skip_if_ySea": 1
                    },
                {
                    "label": "$T_{pla,in}$",
                    "var": "TDisWatRet.T",
                    "linewidth": 0.5,
                    "marker": ">",
                    "color": "k"
                    },
                    {
                    "label": "$T_{pla,out}$",
                    "var": "cenPla.gen.senTemGenLea.T",
                    "linewidth": 0.5,
                    "marker": "<",
                    "color": "k"
                    }
            ]
        },
        {
            "y_label": "Temperature\n[$^\\circ$C]",
            "y_lim": [10, 28],
            "factor": 1,
            "offset": -273.15,
            "vars": [
#                    {
#                    "label": "$T_{bor,per,ret}$",
#                    "var": "cenPla.gen.senTemBorPerRet.T",
#                    "marker": "<",
#                    "linestyle": "--",
#                    "color": "g"
#                    },
#                    {
#                    "label": "$T_{bor,cen,sup}$",
#                    "var": "cenPla.gen.senTemBorCenSup.T",
#                    "linestyle": "-.",
#                    "color": "r",
#                    "marker": ">"
#                    },
#                    {
#                    "label": "$T_{bor,cen,ret}$",
#                    "var": "cenPla.gen.senTemBorCenRet.T",
#                    "linestyle": "-.",
#                    "color": "r",
#                    "marker": "<"
#                    },
                    {
                    "label": "$T_{hea,pum,sup}$",
                    "var": "cenPla.gen.senTemHeaPumEnt.T",
                    "color": "b",
                    "marker": ">"
                    },
                    {
                    "label": "$T_{hea,pum,ret}$",
                    "var": "cenPla.gen.senTemHeaPumLea.T",
                    "color": "b",
                    "marker": "<"
                    }
                ]
            },
#            {
#            "y_label": "Mass flow\nrate [kg/s]",
#            "y_lim": [0, 500],
#            "factor": 1,
#            "offset": 0,
#            "vars": [
#                    {
#                    "label": "$\dot m_{eco}$",
#                    "var": "cenPla.gen.hex.m2_flow",
#                    },
#                    {
#                    "label": "$\dot m_{bor,per}$",
#                    "var": "cenPla.gen.senTemBorPerRet.port_a.m_flow",
#                    "marker": "o",
#                    },
#                    {
#                    "label": "$\dot m_{bor,cen}$",
#                    "var": "cenPla.gen.senTemBorCenRet.port_a.m_flow",
#                    "marker": "x",
#                    },
#                    {
#                    "label": "$\dot m_{hea,pum}$",
#                    "var": "cenPla.gen.heaPum.m1_flow",
#                    "marker": "v",
#                    }
#                    ]
#            },
            {
            "y_label": "Heat flow\nrate [MW]",
            "y_lim": [-8, 8],
            "factor": 1E-6,
            "offset": 0,
            "vars": [
#                    {
#                    "label": "$\dot Q_{eco}$",
#                    "var": "cenPla.gen.hex.Q2_flow",
#                    },
                    {
                    "label": "$\dot Q_{bor,per}$",
                    "var": "cenPla.borFie.QPer_flow",
                    "marker": "o",
                    },
                    {
                    "label": "$\dot Q_{bor,cen}$",
                    "var": "cenPla.borFie.QCen_flow",
                    "marker": "x",
                    },
                    {
                    "label": "$\dot Q_{hea,pum}$",
                    "var": "cenPla.gen.heaPum.Q1_flow",
                    "marker": "v",
                    }
                    ]
        }
    ]

con.plotPlant(lis, r_base, "plant", days)

# Plot borefield energy

In [10]:
# List of days to plot
days = [
    {
        "xlim": [0, 365],
        "date": "",
        "name": "Annual"
    }
]
# List of variables to plot for each subplot
lis = [
        {
            "y_label": "Borefield energy [GWh]",
            "y_lim": [-3, 3],
            "factor": 1/3600/1E9,
            "offset": 0,
            "vars": [
                {
                    "label": "Modelica $E_{bor}$",
                    "var": "EBor.y",
                    "linewidth": 1
                    },
                    {
                    "label": "Modelica $E_{bor,per}$",
                    "var": "EBorPer.y",
                    "linewidth": 1
                    },
                    {
                    "label": "Modelica $E_{bor,cen}$",
                    "var": "EBorCen.y",
                    "linewidth": 1
                    },
                    {
                    "label": "MILP $E_{bor}$",
                    "var": "borMil.E",
                    "linestyle": "--",
                    "mark_every": 50
                    }
                ]
        }
]

con.plotOneFigure(lis, r_base, "borefieldEnergy", days)